# model_train notebook

The model_training notebook uses the 7th dataset file of the IEEE FLAME dataset to traing a classification model able to identify aerial views of fire and no fire.

https://ieee-dataport.org/open-access/flame-dataset-aerial-imagery-pile-burn-detection-using-drones-uavs




In [1]:
# Import libraries

import os.path
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [2]:
# Define model parameters

# Add random transformations to training images to slow down overfitting due to
# smaller dataset
data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
        ]
    )

new_size = {'width': 256, 'height': 256}
Config_classification = {'batch_size': 32, 'Save_Model': True, 'Epochs': 40,
                         'TrainingPlot': True}

# Define size of images, batch size, and # of epochs
image_size = (new_size.get('width'), new_size.get('height'))
batch_size = Config_classification.get('batch_size')
save_model_flag = Config_classification.get('Save_Model')
epochs = Config_classification.get('Epochs')

# Create metrics for visuals
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.Accuracy(name='accuracy'),
    keras.metrics.BinaryAccuracy(name='bin_accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc')
]

Train a DNN model based on Keras and Tensorflow as a backend. At first, the directory of Fire and Non_Fire images should be defined for the model, then the model is defined, compiled and fitted over the training and validation set. At the end, the models is saved based on the *.h5 parameters and weights. Training accuracy and loss are demonstrated at the end of this function.

https://keras.io/examples/vision/image_classification_from_scratch/

In [ ]:
dir_fire = 'Frames/Training/Fire/'
dir_no_fire = 'Frames/Training/No_Fire/'